# Notes from Google Pair Programming LLM Course

References:

* [Deepleaning.AI Course](https://learn.deeplearning.ai/pair-programming-llm/)
* [PaLM API](https://developers.generativeai.google/)

Introduction with PALM API


In [ ]:
import os
from utils import get_api_key
import google.generativeai as palm
from google.api_core import client_options as client_options_lib

palm.configure(
    api_key=get_api_key(),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)
# Pick the model that generates text
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model_bison = models[0]
model_bison
# Helper function to call the PALM API
from google.api_core import retry
@retry.Retry()
def generate_text(prompt, 
                  model=model_bison, 
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)

## Priming the Promt

#### Prompt template

1. priming: getting the LLM ready for the type of task you'll ask it to do.
2. question: the specific task.
3. decorator: how to provide or format the output.

In [ ]:
prompt_template = """
{priming}

{question}

{decorator}

Your solution:
"""

In [ ]:
priming_text = "You are an expert at writing clear, concise, Python code."

In [ ]:
question = "create a doubly linked list"

#### Observe how the decorator affects the output
- In other non-coding prompt engineering tasks, it's common to use "chain-of-thought prompting" by asking the model to work through the task "step by step".
- For certain tasks like generating code, you may want to experiment with other wording that would make sense if you were asking a developer the same question.

In the code cell below, try out option 1 first, then try out option 2.

In [ ]:
# option 1
# decorator = "Work through it step by step, and show your work. One step per line."

# option 2
decorator = "Insert comments for each line of code."

In [ ]:
prompt = prompt_template.format(priming=priming_text,
                                question=question,
                                decorator=decorator)
# Review 
print(prompt)

Then we can just try different questions such as:

In [ ]:
question = """create a very large list of random numbers in python, 
and then write code to sort that list"""
prompt = prompt_template.format(priming=priming_text,
                                question=question,
                                decorator=decorator)
print(prompt)

Pair Programming scenarios

* Improve existing code
* Simplify code
* Write test cases
* Make code more efficient
* Debug your code

### Scenario 1: Improve existing code
- An LLM can help you rewrite your code in the way that's recommended for that particular language.
- You can ask an LLM to rewrite your Python code in a way that is more 'Pythonic".

In [ ]:
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

Please explain, in detail, what you did to improve it.
"""

question = """
def func_x(array)
  for i in range(len(array)):
    print(array[i])
"""

completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.result)

One possible answer could be
```python
def func_x(array):
  print(*array)
```

I improved the code by using the `*` operator to unpack the array into individual arguments for the `print()` function. This is more concise and efficient than using a `for` loop.

Other ways to ask:

In [ ]:
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

Please explore multiple ways of solving the problem, and explain each.
"""
# A more "Pythonic"
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

Please explore multiple ways of solving the problem, 
and tell me which is the most Pythonic
"""

### Scenario 2: Simplify code
- Ask the LLM to perform a code review.
- Note that adding/removing newline characters may affect the LLM completion that gets output by the LLM.

In [ ]:
# option 1
prompt_template = """
Can you please simplify this code for a linked list in Python?

{question}

Explain in detail what you did to modify it, and why.
"""

After you try option 1, you can modify it to look like option 2 (in this markdown cell) and see how it changes the completion.
```Python
# option 2
prompt_template = """
Can you please simplify this code for a linked list in Python? \n
You are an expert in Pythonic code.

{question}

Please comment each line in detail, \n
and explain in detail what you did to modify it, and why.
"""
```

In [ ]:
question = """
class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.headval = None

list1 = SLinkedList()
list1.headval = Node("Mon")
e2 = Node("Tue")
e3 = Node("Wed")
list1.headval.nextval = e2
e2.nextval = e3

"""
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.result)

### Scenario 3: Write test cases

- It may help to specify that you want the LLM to output "in code" to encourage it to write unit tests instead of just returning test cases in English.

In [ ]:
prompt_template = """
Can you please create test cases in code for this Python code?

{question}

Explain in detail what these test cases are designed to achieve.
"""
# Note that the code I'm using here was output in the previous
# section. Your output code may be different.
question = """
class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.head = None

def create_linked_list(data):
  head = Node(data[0])
  for i in range(1, len(data)):
    node = Node(data[i])
    node.nextval = head
    head = node
  return head

list1 = create_linked_list(["Mon", "Tue", "Wed"])
"""
completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.result)

### Scenario 4: Make code more efficient
- Improve runtime by potentially avoiding inefficient methods (such as ones that use recursion when not needed).

In [ ]:
prompt_template = """
Can you please make this code more efficient?

{question}

Explain in detail what you changed and why.
"""

question = """
# Returns index of x in arr if present, else -1
def binary_search(arr, low, high, x):
    # Check base case
    if high >= low:
        mid = (high + low) // 2
        if arr[mid] == x:
            return mid
        elif arr[mid] > x:
            return binary_search(arr, low, mid - 1, x)
        else:
            return binary_search(arr, mid + 1, high, x)
    else:
        return -1

# Test array
arr = [ 2, 3, 4, 10, 40 ]
x = 10

# Function call
result = binary_search(arr, 0, len(arr)-1, x)

if result != -1:
    print("Element is present at index", str(result))
else:
    print("Element is not present in array")

"""

completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.result)

### Scenario 5: Debug your code

In [ ]:
prompt_template = """
Can you please help me to debug this code?

{question}

Explain in detail what you found and why it was a bug.
"""

# Lesson 4: Technical Debt

Complex code being handle down from developer to developer over time, it was writing long time ago, lots of dependencies and hard to change. Here is where LLMs also can help how to understand the code to be able to use it properly.

### Ask an LLM to explain a complex code base

In [ ]:
#@title Complex Code Block
# Note: Taken from https://github.com/kevinmgamboa/consciousness/blob/main/train.py
CODE_BLOCK = """
"""
Train a model on the Sleep Dataset
Created on Wed Jun  9 21:18:16 2021
@author: kevin machado gamboa
"""
# -----------------------------------------------------------------------------
#                           Libraries Needed
# -----------------------------------------------------------------------------
import os
import copy
import numpy as np
import pandas as pd

# Libraries for training process
from sklearn.model_selection import KFold
# ML library
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Personal Libraries
from helpers_and_functions import config, main_functions as mpf, utils
import modelhub as mh
import databases as dbs

# %%
# ------------------------------------------------------------------------------------
#                               Loading sleep dataset
# ------------------------------------------------------------------------------------
# initialize sleep database
sleep = dbs.sleep()
# loads [x_epochs, y_labels]
sleep.load_epochs_labels(t_files=5, n_test=0.30)
# converts labels to [0=>conscious,5* 1=>unconscious]
sleep.get_binary_labels()
# Normalize the dataset between [-1,1]
sleep.transform(mpf.nor_dataset)
# applying dataset transformation e.g. 'spectrogram'
sleep.transform(mpf.raw_chunks_to_spectrograms, name='spectrogram')
# make dataset ready for training
sleep.get_ready_for_training()

# %%
# ------------------------------------------------------------------------------------
#                                Cross-Validation
# ------------------------------------------------------------------------------------
# Creates folder to store experiment
date = utils.create_date()
os.mkdir('log_savings/sleep_' + date)
# confusion matrix per fold variable
cm_per_fold = []
# number of train epochs
train_epochs = 30
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
#                                               verbose=1, restore_best_weights=True)
kfold = KFold(n_splits=config.NUM_FOLDS, shuffle=True)
# init fold number
fn = 1

# stores the models with best acc & other info
model_best = {'model': [],
          'score': [],
          'tra_with': [],
          'val_with': [],
          'train_history': [],
          'initial_weights': [],
          'test_acc_per_fold': [],
          'test_loss_per_fold': [],
          'transformation': sleep.info['transformation']}

# training parameters
parameters = {'lr': 1e-6,
          'num_filters': 10,
          'kernel_size': 3,
          'dense_units': 10,
          'out_size': 1}

p_count = 0  # early stopping counter
patient = 5  # wait n epochs for error to keep decreasing, is not stop

all_folds_best_test_score = 0.0
for tra, val in kfold.split(sleep.data['train']['epochs'], sleep.data['train']['labels']):
    # Call the hub
    hub = mh.simple_cnn_2(param=parameters)
    # build model structure
    hub.build_model_structure(sleep.info['data_shape'])
    # compile model
    hub.compile()
    # initializing model
    model_best_fold = tf.keras.models.clone_model(hub.model)
    # initial model weights
    ini_wei = hub.model.get_weights()
    # initializing training history
    train_history = []
    # defines an initial score
    pre_score = [1.0, 0.0]
    # Generate a print
    print(100 * '-')
    print(f'--------------------------------- Training for fold {fn} ---------------------------------')
    # -----------------------------------------------------------------------------
    #                               Train-Test Loop
    # -----------------------------------------------------------------------------
    for n_ep in range(train_epochs):
        print('------- train score -------')
        # Train the model
        train_score = hub.model.pdf(sleep.data['train']['epochs'][tra],
                                    sleep.data['train']['labels'][tra],
                                    validation_data=(sleep.data['train']['epochs'][val],
                                                     sleep.data['train']['labels'][val]),
                                    epochs=1)#, callbacks=[early_stop])
        print('------- test score -------')
        # Evaluates on Test set
        test_scores = hub.model.evaluate(sleep.data['test']['epochs'], sleep.data['test']['labels'])
        # saving train history
        train_score = list(np.concatenate(list(train_score.history.values())))
        # Adding test score to train score
        train_score.extend(test_scores)
        # Train history including the score in test set
        train_history.append(train_score)

        # Stores the best model in the fold
        if test_scores[1] > pre_score[1]:
            print(f'new best score in the fold: {test_scores[1]:.4}')
            # saves best model INSIDE FOLD
            hub.model.save('log_savings/sleep_' + date + '/best_fold_model.h5')
            # Saves best model from ALL FOLDS
            if test_scores[1] > all_folds_best_test_score:
                print(f'new best model from ALL FOLDS {test_scores[1]:.4} ')
                all_folds_best_test_score = test_scores[1]
                # saves best model
                hub.model.save('log_savings/sleep_' + date + '/all_folds_best_model.h5')
            # updating previous score
            pre_score = copy.copy(test_scores)
            # reset the stopping patient counter
            p_count = 0
        else:  # Stopping criteria:
            p_count += 1
            if p_count >= patient:
                print('Early Stopping !!! Error hasnt decreased')
                p_count = 0
                break
    # -----------------------------------------------------------------------------
    #                          Stores Data from Each Fold
    # -----------------------------------------------------------------------------
    # save train history
    model_best['train_history'].append(train_history)
    # save best score from fold
    train_history = pd.DataFrame(train_history)
    idx = train_history[5].idxmax()  # max idx test acc
    model_best['score'].append(train_history[5][idx])
    # saves segments of data the model was trained with
    model_best['tra_with'].append(tra)
    model_best['val_with'].append(val)
    # save model initial weights
    model_best['initial_weights'].append(ini_wei)

    print(
        f'Best score fold {fn}: {hub.model.metrics_names[0]}: {train_history[4][idx]:.4}; {hub.model.metrics_names[1]}: {train_history[5][idx] * 100:.4}%')
    # Adds test score
    model_best['test_acc_per_fold'].append(train_history[5][idx] * 100)
    model_best['test_loss_per_fold'].append(train_history[4][idx])
    # confusion matrix per fold
    # -------------------------
    # Load best model in fold
    model_best_fold = tf.keras.models.load_model('log_savings/sleep_' + date + '/best_fold_model.h5')
    # Confusion matrix of best model in fold
    cm_per_fold.append(utils.get_confusion_matrix(model_best_fold, sleep.data['test'],
                                                  sleep.info['class_balance']['test']['value']))
    # Increase fold number
    fn += 1
# remove model per fold
os.remove('log_savings/sleep_' + date + '/best_fold_model.h5')
#%%
# ------------------------------------------------------------------------------------
#                                    Final Results
# ------------------------------------------------------------------------------------
# confusion matrix dataframe across participants
df = utils.cm_fold_to_df(cm_per_fold, model_best['test_loss_per_fold'])
utils.boxplot_evaluation_metrics_from_df(df, x_axes='fold')

# plots train history for the best model
utils.plot_train_test_history(model_best)

# Plots the confusion matrix of the best model the folds
cm_categories = {0: 'Conscious', 1: 'Unconscious'}
labels = [' True Pos', ' False Neg', ' False Pos', ' True Neg']
utils.make_confusion_matrix(cm_per_fold[np.argmax(model_best['score'])], group_names=labels, categories=cm_categories,
                            class_balance=sleep.info['class_balance']['test']['value'],
                            title='Confusion Matrix of Best Model')
#%%
# ------------------------------------------------------------------------------------
#                                       Savings
# ------------------------------------------------------------------------------------
# import json

df.to_csv('log_savings/sleep_' + date + '/folds.csv')
np.save('log_savings/sleep_' + date + '/model_best.npy', model_best)
# with open('log_savings/sleep_' + date + '/best_model.json', 'wb') as file:
#     file.write(json.dumps(model_best).encode("utf-8"))
    #json.dump(model_best, file, indent=4)


# utils.super_test(tf.keras.models.load_model('log_savings/sleep_' + date + '/all_folds_best_model.h5'),
#                  feature_function, dataset='anaesthesia')
# #%%
# # ------------------------------------------------------------------------------------
# #                                    Compares with Benchmark
# # ------------------------------------------------------------------------------------
# # check if current scores overpasses benchmark
# utils.check_benchmark(model_best, database='sleep')

"""

In [ ]:
prompt_template = """
Can you please explain how this code works?

{question}

Use a lot of detail and make it as clear as possible.
"""
completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)
print(completion.result)

### Ask an LLM to document a complex code base


In [ ]:
prompt_template = """
Please write technical documentation for this code and \n
make it easy for a non swift developer to understand:

{question}

Output the results in markdown
"""
completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)
print(completion.result)